### History Matching
#### LAI pxb & GPP biome
#### Wave 1 : February 2024

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import tensorflow as tf
import sys

2024-02-21 09:14:37.318939: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-21 09:14:37.880251: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-21 09:14:38.205467: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /glade/u/apps/casper/23.10/spack/opt/spack/openmpi/4.1.6/oneapi/2023.2.1/dgcv/lib:/glade/u/apps/

In [2]:
### import some analysis functions we wrote for this project
#import sys ; sys.path.append("../") ; sys.path.append("../../") 
#from ppe_analysis.analysis import *
from pyfunctions import *

In [3]:
#############################################
# Setup
whit=xr.open_dataset('./whit/whitkey.nc')
biome_names = whit.biome_name.values
pfts=xr.open_dataset('/glade/campaign/asp/djk2120/PPEn11/paramfiles/OAAT0000.nc').pftname
pft_names=[str(p)[2:-1].strip() for p in pfts.values]

nbiomes = 10
npfts = 17
pft_ids = [1,2,3,4,5,6,7,8,10,11,12,13,14,15]
biome_ids = [1,3,5,6,7,8]
pb_index = np.load('pb_index.npy',allow_pickle='TRUE').item()
SP_LAI_max = xr.open_dataset("CLM-SP_LAI_pxb_amax_2003-2007.nc")
SP_LAI_mean = xr.open_dataset("CLM-SP_LAI_pxb_amean_2003-2007.nc")
LAI_var = pd.read_csv("LAI_amax_bmean_variance.csv",sep=',')

In [4]:
# load LHC PPE parameter sets 
lhckey = '/glade/campaign/asp/djk2120/PPEn11/csvs/lhc220926.txt'
df = pd.read_csv(lhckey)
lhc_params = df.drop(columns='member')
num_params = len(lhc_params.columns)

pft_params   = ['froot_leaf','kmax','krmax','leaf_long','leafcn','lmr_intercept_atkin','medlynslope','medlynintercept','psi50','slatop','stem_leaf','theta_cj']
pftix=np.array([p in pft_params for p in lhc_params])

In [6]:
#############################################
# build sample

n_usamp = 100
u_sample = np.random.rand(n_usamp,20)

n_psamp = 1000
p_sample = np.random.rand(n_psamp,12)

sample=np.zeros([n_usamp*n_psamp,num_params])
sample[:,~pftix]= np.repeat(u_sample,n_psamp,axis=0)
sample[:,pftix]= np.tile(p_sample,(n_usamp,1))

#np.savetxt("./example/tmp_sample.txt", sample, delimiter=',')

In [7]:
%%time

#############################################
# Calculate implausibility

uset_I = np.empty((n_usamp,npfts))*np.NaN
set_I = np.empty((n_usamp*n_psamp,npfts,nbiomes))*np.NaN

pred = np.empty((n_usamp*n_psamp,npfts,nbiomes))*np.NaN
pred_var = np.empty((n_usamp*n_psamp,npfts,nbiomes))*np.NaN

for p in pft_ids:
    I_tmp = np.empty((n_usamp,n_psamp,nbiomes))*np.NaN
    
    for b in pb_index[pft_names[p]]:  

        loaded_emulator = tf.saved_model.load('/glade/u/home/linnia/clm5ppe/pyth/wave1/models_lai/pft'+str(p)+'_biome'+str(b))
        y, y_var = loaded_emulator.predict(sample)
        
        pred[:,p,b] = y.numpy().flatten()
        pred_var[:,p,b] = y_var.numpy().flatten()

        pxb = p*10+b
        if (p < 12): # not grass
            o = SP_LAI_max.TLAI.sel(pxb=pxb).values[0]
        else: # grass
            o = SP_LAI_mean.TLAI.sel(pxb=pxb).values[0]
        ovar = LAI_var.TLAI[b]

        I = np.abs(o-y) / np.sqrt(ovar + y_var)
        set_I[:,p,b] = I[:,0]
        I_tmp[:,:,b] = I.reshape((n_usamp,n_psamp))
        
    ix = I_tmp.copy()
    ix[ix<3] = 1
    ix[ix>=3] = 0
    x = np.nanmean(ix,axis=2)
    uset_I[:,p] = np.any(x==1,axis=1)

uset_I[:,9] = np.NaN # ignore this PFT

#np.save("/glade/work/linnia/CLM-PPE-LAI_tests/exp5_PXB/predicted/PxB_LAI_I_sample"+str(idx)+".npy", set_I)
#np.save("/glade/work/linnia/CLM-PPE-LAI_tests/exp5_PXB/predicted/PxB_LAI_pred_sample"+str(idx)+".npy", pred)
#np.save("/glade/work/linnia/CLM-PPE-LAI_tests/exp5_PXB/predicted/PxB_LAI_predvar_sample"+str(idx)+".npy", pred_var)


2024-02-20 21:50:40.351927: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-02-20 21:50:40.351987: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (crhtc53): /proc/driver/nvidia/version does not exist
2024-02-20 21:50:40.352532: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


CPU times: user 4min 21s, sys: 31.4 s, total: 4min 52s
Wall time: 5min 27s


In [8]:
#####################################################
# identify universal sets that have PFT sets in NROY for all PFTs
ix_usets = np.where(np.nanmean(uset_I,axis=1)==1)[0]

In [62]:
bool_usets = (np.nanmean(uset_I,axis=1)==1) 

In [63]:
#####################################################
# select one PFT set per Universal set (for each PFT)
sample_NROY = np.empty((len(ix_usets),num_params,npfts))*np.NaN
for pft in pft_ids:
    bool_pft_biome = np.empty((len(ix_usets),nbiomes,n_psamp))*np.NaN # index of NROY sets for each PFT each Biome
    for b in pb_index[pft_names[pft]]:
        v = np.reshape(set_I[:,pft,b],(n_usamp,n_psamp))
        bool_pft_biome[:,b,:] = v[ix_usets,:]<3
    
    ix_set = []
    for u in range(len(ix_usets)):        
        ix_all_psets = np.where(np.nanmean(bool_pft_biome,axis=1)[u,:]==1)[0]
        ix_pset = np.random.choice(ix_all_psets, size= 1,replace=False)[0]
        
        ix_set.append(ix_usets[u]*n_psamp + ix_pset)
        
    sample_NROY[:,:,pft] = sample[ix_set,:]
    
#######################################################    
# Save plausible sets    
#np.save("/glade/work/linnia/CLM-PPE-LAI_tests/wave1/selected/PxB_NROYsets_sample"+str(idx)+".npy", sample_NROY)


In [70]:
sample_NROY[0,:,9]

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan])

In [ ]:
## Check
# Emulator prediction of selected sample:
best_samples = sample_NROY
best_pred = np.empty([np.shape(best_samples)[0],npfts,nbiomes])*np.NaN
best_pred_var = np.empty([np.shape(best_samples)[0],npfts,nbiomes])*np.NaN

for p in [1,2,3,4,5,6,7,8,10,11,12,13,14,15]:
    sample = sample_all[:,:,p]
    for b in pb_index[pft_names[p]]:  

        loaded_emulator = tf.saved_model.load('/glade/u/home/linnia/clm5ppe/pyth/wave/models_lai/pft'+str(p)+'_biome'+str(b))
        y, y_var = loaded_emulator.predict(best_samples[:,:,p])
        
        best_pred[:,p,b] = y.numpy().flatten()
        best_pred_var[:,p,b] = y_var.numpy().flatten()

#### GPP

In [5]:
GPP = pd.read_csv("FLUXCOM-GSWP3_GPP_amean_bmean_2001-2010.csv")
GPP_var = pd.read_csv("FLUXCOM_GPP_amean_bmean_variance.csv")
bp_index = np.load('bp_index.npy', allow_pickle=True).item()

In [6]:
# function to calculate total Biome GPP from PFT contributions
f='/glade/work/djk2120/ppe_savs/pbmean/sg_lapxb.nc'
lapxb=xr.open_dataset(f).lapxb
ix=(lapxb.pxb>9)&(lapxb.mean(dim='year').sum(dim='pft')>0)
pxb_la = lapxb.isel(pxb=ix).sum(dim='pft')
pb_la = pxb_la.sel(year=slice(2001,2010)).mean(dim='year').compute()

whit = xr.open_dataset('../whit/whitkey.nc')
la = get_la()
biome_la = la.groupby(whit.biome).sum()

In [12]:
sample = np.loadtxt("/glade/work/linnia/CLM-PPE-LAI_tests/wave1.1/samples/PxB_sample_001.txt", delimiter=',')

In [13]:
#############################################
# Calculate implausibility GPP

uset_I = np.empty((n_usamp,nbiomes))*np.NaN
set_I = np.empty((n_usamp*n_psamp,nbiomes))*np.NaN

pred = np.empty((n_usamp*n_psamp,npfts,nbiomes))*np.NaN
pred_var = np.empty((n_usamp*n_psamp,npfts,nbiomes))*np.NaN

data = GPP

for b in biome_ids:
    o = GPP.GPP[b]
    ovar = GPP_var.GPP[b]

    I_tmp = np.empty((n_usamp,n_psamp,nbiomes))*np.NaN
    
    for i,p in enumerate(bp_index[biome_names[b]]):  
        pxb = p*10+b
        la_p = pb_la.sel(pxb=pxb).values
        
        loaded_emulator = tf.saved_model.load('/glade/u/home/linnia/clm5ppe/pyth/wave1/models_gpp/pft'+str(p)+'_biome'+str(b))
        y, y_var = loaded_emulator.predict(sample)
        
        pred[:,p,b] = y.numpy().flatten()
        pred_var[:,p,b] = y_var.numpy().flatten()

        if (i ==0):
            y_pred_all = la_p*y.numpy().flatten()
            y_pred_var_all = la_p*y_var.numpy().flatten()

        else:
            y_pred_all = np.column_stack((y_pred_all,la_p*y.numpy().flatten()))
            y_pred_var_all = np.column_stack((y_pred_var_all,la_p*y_var.numpy().flatten()))

    y_pred_biome = np.sum(y_pred_all,axis=1)*(1/biome_la[b].values)
    y_pred_var_biome = np.sum(y_pred_var_all,axis=1)*(1/biome_la[b].values)
        
    I = np.abs(o-y_pred_biome) / np.sqrt(ovar + y_pred_var_biome)
    set_I[:,b] = I
    I_tmp[:,:,b] = I.reshape((n_usamp,n_psamp))
        
    ix = I_tmp.copy()
    ix[ix<3] = 1
    ix[ix>=3] = 0
    x = np.nanmean(ix,axis=2)
    uset_I[:,b] = np.any(x==1,axis=1)

#np.save("/glade/work/linnia/CLM-PPE-LAI_tests/wave1/predicted/PxB_GPP_I_sample"+str(idx)+".npy", set_I)
#np.save("/glade/work/linnia/CLM-PPE-LAI_tests/wave1/predicted/PxB_GPP_pred_sample"+str(idx)+".npy", pred)
#np.save("/glade/work/linnia/CLM-PPE-LAI_tests/wave1/predicted/PxB_GPP_predvar_sample"+str(idx)+".npy", pred_var)


In [14]:
#####################################################
# identify universal sets that have PFT sets in NROY for all Biomes
ix_usets = np.where(np.nanmean(uset_I,axis=1)==1)[0]

In [15]:
np.shape(ix_usets)

(100,)